## MBAN 6110 Assignment 3

Questions / Tasks:
1. Build a regression model to predict the price of a house. You may choose between Ordinary Least Squares Regression, LASSO, Ridge or Elastic Net.

2. Evaluate the model using techniques covered in class and explain the results. How do you know this is the best model you can build, given the tools you have?

3. Explain the results to a business executive. What are the main drivers of house prices in King City? And how much do these drivers impact the price?

In [2]:
# import libraries and packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.metrics import mean_squared_error

In [3]:
dt = pd.read_csv('/Users/jiawenli/Desktop/MBAN_6110T/Assignment 3/kc_house_data.csv')

### Data Cleaning and transformation

In [4]:
dt.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [5]:
dt.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [6]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

In [7]:
# id is a unique identifier. We can set id as index column,  as id does not provide any insight for our model. 
dt = dt.set_index('id')

Convert data type of date column into integer

In [8]:
# Keep only the first eight digits of the date column, and remove the trailing suffix like ‘T000000’
dt['date'] = dt['date'].astype(str).str.slice(0,8)

In [9]:
# Convert data type of date column into integer
dt['date'] = dt['date'].astype(int)

Check for missing value

In [10]:
dt.isnull().sum()

date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

We do not have any missing value for the dataset. This is good.

Then, we need to check for duplicates

In [11]:
# delete duplicated rows
dt = dt.drop_duplicates()

In [12]:
# create another clean copy of the dataframe dt for future use, name it dt_copy
copied_dt = dt.copy()

Now, we can generate dummy variables from the 'grade' column. We can concatenate the dummies with our original dataframe.

The first step is perform feature engineering. 
We could categorize 1-3 as low quality, 4-6 as sub-moderate quality, 7 as average for architecture and design, 8-10 as sub-high quality, and 11-13 as high quality level for architecture and design.

In [13]:
def application_1(d):
    if d >= 1 and d <= 3:
        return 'low quality'
    elif d >= 4 and d <= 6:
        return 'sub_moderate quality'
    elif d == 7:
        return 'average quality'
    elif d >= 8 and d <= 10:
        return 'sub_high quality'
    else:
        return 'high quality'

dt['grade'] = dt['grade'].apply(application_1)

The next step is to craete dummies for the adjusted new 'grade' column.

In [14]:
grade_dummies = pd.get_dummies(dt['grade'])

In [15]:
grade_dummies.head()

,average quality,high quality,low quality,sub_high quality,sub_moderate quality
id,,,,,
7129300520,1,0,0,0,0
6414100192,1,0,0,0,0
5631500400,0,0,0,0,1
2487200875,1,0,0,0,0
1954400510,0,0,0,1,0


In [16]:
dt = pd.concat([dt,grade_dummies], axis=1)

In [17]:
dt.head(3)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,average quality,high quality,low quality,sub_high quality,sub_moderate quality
id,,,,,,,,,,,,,,,,,,,,,
7129300520,20141013,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,98178,47.5112,-122.257,1340,5650,1,0,0,0,0
6414100192,20141209,538000.0,3,2.25,2570,7242,2.0,0,0,3,...,98125,47.7210,-122.319,1690,7639,1,0,0,0,0
5631500400,20150225,180000.0,2,1.00,770,10000,1.0,0,0,3,...,98028,47.7379,-122.233,2720,8062,0,0,0,0,1


Then, the last thing we are going to do is to drop latitude and longitude. We only keep zipcode in our dataframe. Zipcode and latitude, longitude actually represent the same thing in our data. They both refer to the location and lot of the house. In this case, we just need to keep one.

In [18]:
dt.drop('lat', axis=1, inplace=True)
dt.drop('long',axis=1, inplace=True)

In [19]:
dt.head(3)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15,average quality,high quality,low quality,sub_high quality,sub_moderate quality
id,,,,,,,,,,,,,,,,,,,,,
7129300520,20141013,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,1955,0,98178,1340,5650,1,0,0,0,0
6414100192,20141209,538000.0,3,2.25,2570,7242,2.0,0,0,3,...,1951,1991,98125,1690,7639,1,0,0,0,0
5631500400,20150225,180000.0,2,1.00,770,10000,1.0,0,0,3,...,1933,0,98028,2720,8062,0,0,0,0,1


### Build a regression model to predict the price of a house

I will be focusing on both categorical features and numerical feature.

The numerical features contains bedrooms, bathrooms, sqft_living, sqft_lot, floors.

The categorical features contains waterfront, view, condition, grade.

In [20]:
# Set up categorical features and numerical features.
# In addition, we also set up target variable
numerical_feature = ['bedrooms','bathrooms','sqft_living','sqft_lot','floors']
categorical_feature = ['waterfront','view','condition', 'low quality','sub_moderate quality',
                       'average quality', 'sub_high quality','high quality']
target_variable = ['price']

In [21]:
dt[numerical_feature + categorical_feature].head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,low quality,sub_moderate quality,average quality,sub_high quality,high quality
id,,,,,,,,,,,,,
7129300520,3,1.00,1180,5650,1.0,0,0,3,0,0,1,0,0
6414100192,3,2.25,2570,7242,2.0,0,0,3,0,0,1,0,0
5631500400,2,1.00,770,10000,1.0,0,0,3,0,1,0,0,0
2487200875,4,3.00,1960,5000,1.0,0,0,5,0,0,1,0,0
1954400510,3,2.00,1680,8080,1.0,0,0,3,0,0,0,1,0


In [22]:
# Then, we need to split our data into training and testing using columns in numerical and categorical features
X_train, X_test, y_train, y_test = train_test_split(dt[numerical_feature + categorical_feature], 
                                                    dt[target_variable], test_size=0.3, random_state=1234)

In [23]:
X_train

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,low quality,sub_moderate quality,average quality,sub_high quality,high quality
id,,,,,,,,,,,,,
1323089056,2,1.75,1620,113862,1.5,0,0,3,0,0,1,0,0
1395500020,3,1.00,1400,10800,1.0,0,0,3,0,1,0,0,0
8024200010,2,1.00,1460,6000,1.0,0,0,2,0,0,1,0,0
3821000100,4,1.75,1620,10530,1.0,0,0,3,0,0,1,0,0
1422029138,3,2.50,2030,217805,1.0,0,0,3,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1954700610,3,2.25,3360,7108,2.0,0,0,3,0,0,0,1,0
3472800065,4,3.00,3600,9687,2.0,0,0,4,0,0,0,1,0
714000315,3,2.75,1710,9448,1.0,0,0,3,0,0,1,0,0


In [24]:
X_test

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,low quality,sub_moderate quality,average quality,sub_high quality,high quality
id,,,,,,,,,,,,,
9414610240,3,1.75,2030,10089,1.0,0,0,4,0,0,0,1,0
251300110,3,2.25,2510,12013,2.0,0,0,3,0,0,0,1,0
7937900220,5,2.75,3630,38461,2.0,0,0,3,0,0,0,0,1
1771110640,3,1.00,1660,11783,1.0,0,0,4,0,0,1,0,0
3905100310,4,2.50,2030,3974,2.0,0,0,3,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8056000075,2,1.75,1890,9466,2.0,0,0,3,0,0,0,1,0
8564860280,3,2.50,2680,5539,2.0,0,0,3,0,0,0,1,0
5137800130,4,2.50,2718,6197,2.0,0,0,3,0,0,0,1,0


In [25]:
y_train

,price
id,
1323089056,439000.0
1395500020,279900.0
8024200010,312000.0
3821000100,249950.0
1422029138,565000.0
...,...
1954700610,2193000.0
3472800065,1698000.0
714000315,515000.0


----------------------

#####  Building regression models using Ordinary Least Squares, LASSO, Ridge and Elastic Net

The purpose of doing them all is to compare them all and find a better model that suits the best with our purpose given what we have as our insights.

In [26]:
dt.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15,average quality,high quality,low quality,sub_high quality,sub_moderate quality
id,,,,,,,,,,,,,,,,,,,,,
7129300520,20141013,221900.0,3,1.00,1180,5650,1.0,0,0,3,...,1955,0,98178,1340,5650,1,0,0,0,0
6414100192,20141209,538000.0,3,2.25,2570,7242,2.0,0,0,3,...,1951,1991,98125,1690,7639,1,0,0,0,0
5631500400,20150225,180000.0,2,1.00,770,10000,1.0,0,0,3,...,1933,0,98028,2720,8062,0,0,0,0,1
2487200875,20141209,604000.0,4,3.00,1960,5000,1.0,0,0,5,...,1965,0,98136,1360,5000,1,0,0,0,0
1954400510,20150218,510000.0,3,2.00,1680,8080,1.0,0,0,3,...,1987,0,98074,1800,7503,0,0,0,1,0


##### 1. First, we are going to fit a model using Ordinary Least Square

In [27]:
# First, we are going to fit a model using Ordinary Least Square
ols_model = ols("price ~ bedrooms + bathrooms + sqft_living + floors + waterfront + view + condition + grade", data=dt).fit()
print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.600
Model:                            OLS   Adj. R-squared:                  0.600
Method:                 Least Squares   F-statistic:                     2942.
Date:                Sun, 09 Jul 2023   Prob (F-statistic):               0.00
Time:                        18:58:23   Log-Likelihood:            -2.9771e+05
No. Observations:               21613   AIC:                         5.954e+05
Df Residuals:                   21601   BIC:                         5.955e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

From this model, we could see that R-squared number of 0.6 is relatively high.
An R² value of 0.6 means that the model explains 60% of the variation among the data points in the training data set.

Given that an R-square value of 0.6 is particularly high, then it usually means that the model fits the data very well, in other words, the independent variables in the model explain most of the variation in the dependent variable.

However, it is important to note that having 0.6 of r-square in this case, is not always a good thing. 

Because OLS does not include any form of feature selection or regularization, and because we included multiple predictors in our model, this can lead to over-fitting. Overfitting can lead to problem where this model may only work for our training data, but will perform poorly or be biased when we make predictions or and judgments on a completely new data sets.

Thus, we should NOT consider the OLS Model.

----------------------------------------------

##### Before we dive into the other models, we could build a Pipeline for Feature Preprocessing.
We can define a pipeline of two pre-processing steps, we can do this to make our model training more efficient.
1. categorical transformer pipeline:
    
    This step uses OneHotEncoder to encode the categorical variables. This gives us ability to easily convert categorical variables into numerical representations that can be used for model training.

2. numerical transformer pipelines:

    This step uses the MinMaxScaler to normalize numeric variables. MinMaxScalar converts the values of numerical features into a ratio relative to their minimum and maximum values. This normalization method preserves the relative relationship between features in the original data.
    Typically, all data are scaled to between 0 and 1 based on the relative size relationships between the datapoints.

In [28]:
cat_transformer = Pipeline(steps=[('onehot',OneHotEncoder(handle_unknown='ignore'))])
num_transformer = Pipeline(steps=[('scalar', MinMaxScaler())])

In [29]:
# building processor
preprocessor = ColumnTransformer(transformers=[('cat',cat_transformer, categorical_feature),
                                               ('num', num_transformer, numerical_feature)])

##### 2. Secondly, we could try LASSO regression model

In [30]:
# Create Lasso model
lasso_regression = Pipeline(steps= [
    ('preprocessor', preprocessor),
    ('regressor', Lasso(alpha=1.0))
    ])

In [31]:
# Fit the Lasso model
lasso_regression.fit(X_train, y_train)

/Users/jiawenli/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.214e+13, tolerance: 2.071e+11
  model = cd_fast.enet_coordinate_descent(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['waterfront', 'view',
                                                   'condition', 'low quality',
                                                   'sub_moderate quality',
                                                   'average quality',
                                                   'sub_high quality',
                                                   'high quality']),
                                                 ('num',
                                                  Pipeline(steps=[('scalar',
                                                                   MinMaxScaler())]),
                                                  ['bedrooms', 'bathrooms',
                                                   'sqft_living', 'sqft_lot',
                                                   'floors'])])),
                ('regressor', Lasso())])

In [32]:
# Evaluate the model
print("R-square: {:.6f}".format(lasso_regression.score(X_train, y_train)))

R-square: 0.604739


In [33]:
lasso_pred = lasso_regression.predict(X_test)

mse = mean_squared_error(lasso_pred, y_test)

rmse = np.sqrt(mse)

print("Mean Squared Error: {:.6f}".format(mse))
print("Root Mean Squared Error: {:.6f}".format(rmse))

Mean Squared Error: 51925996895.539307
Root Mean Squared Error: 227872.764708


1. Mean Squared Error (MSE) is the average of the squares of the differences between the predicted values and actual values(y_test).

2. RMSE evaluates the root mean square error between the predicted values and actual values(y_test).

##### 3. Then, we could try both Ridge and Elastic Net model

In [34]:
# Fit a Ridge model
ridge_regression = Pipeline(steps= [
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=1.0))
    ])

In [35]:
ridge_regression.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['waterfront', 'view',
                                                   'condition', 'low quality',
                                                   'sub_moderate quality',
                                                   'average quality',
                                                   'sub_high quality',
                                                   'high quality']),
                                                 ('num',
                                                  Pipeline(steps=[('scalar',
                                                                   MinMaxScaler())]),
                                                  ['bedrooms', 'bathrooms',
                                                   'sqft_living', 'sqft_lot',
                                                   'floors'])])),
                ('regressor', Ridge())])

In [39]:
coefficients = ridge_regression.named_steps['regressor'].coef_
print(coefficients)

[[-231742.95766723  231742.95766732 -158461.72954053  -13834.52520639
   -53545.81200031  -12067.49433069  237909.56107825  -27296.97571811
   -30330.99646511  -35912.16507376    7177.18991339   86362.94734349
    47729.25877759  -47729.25877759   79398.89303225  -79398.89303225
    70944.92007554  -70944.92007582   31185.56312796  -31185.5631281
  -229258.63501517  229258.63501521 -891246.01793147   -6139.58852875
  2822088.47328596 -616660.53731859   32605.74351232]]


In [ ]:
# Evaluate the model
print("R-square: {:.6f}".format(lasso_regression.score(X_train, y_train)))

ridge_pred = ridge_regression.predict(X_test)
rmse = np.sqrt(mean_squared_error(ridge_pred,y_test))

print("Root Mean Squared Error: {:.6f}".format(rmse))

R-square: 0.604739
Root Mean Squared Error: 227704.181611


In [ ]:
# Fit an Elastic Net model
elasticnet_regression = Pipeline(steps= [
    ('preprocessor', preprocessor),
    ('regressor', ElasticNet(alpha=1.0, l1_ratio=0.2))
    ])

In [ ]:
elasticnet_regression.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['waterfront', 'view',
                                                   'condition', 'low quality',
                                                   'sub_moderate quality',
                                                   'average quality',
                                                   'sub_high quality',
                                                   'high quality']),
                                                 ('num',
                                                  Pipeline(steps=[('scalar',
                                                                   MinMaxScaler())]),
                                                  ['bedrooms', 'bathrooms',
                                                   'sqft_living', 'sqft_lot',
                                                   'floors'])])),
                ('regressor', ElasticNet(l1_ratio=0.2))])

In [ ]:
# Evaluate the model
print("R-square: {:.6f}".format(elasticnet_regression.score(X_train, y_train)))

elasticnet_pred = elasticnet_regression.predict(X_test)
rmse = np.sqrt(mean_squared_error(elasticnet_pred,y_test))

print("Root Mean Squared Error: {:.6f}".format(rmse))

R-square: 0.162959
Root Mean Squared Error: 328562.418850


To summarize:
Let us see the value of both r-square and root mean squared error for all of the models we have developed.

1. The Lasso Regression Model: 

        R-square value: 0.604739

        Root Mean Squared Error: 227872.764708

2. The Ridge Model:

        R-square: 0.604739

        Root Mean Squared Error: 227704.181611

3. The Elastic Net Model:

        R-square: 0.162959
        
        Root Mean Squared Error: 328562.418850

The smaller the value of RMSE, the better the predictive performance of the model. 

At first glance, we could tell that the Elastic Net Model has the worst performance in terms of a low R-square value and a high Root Mean Square Error.

Then, both of the Lasso regression model and the Ridge model have a R-square value of 0.6047. This is perfect. However, the Root Mean Square Error of Ridge is the slightly lower with a value of about 227704.

##### To conclude, we would choose the Ridge Model as it has the best performance.

-----------------------

### Figure out the main drivers of house prices in King City using a simple Ridge model? Identify how much do these drivers impact the price?

In [ ]:
ridge_reg = Ridge(alpha=1.0)

ridge_reg.fit(X_train, y_train)

coefficients = ridge_reg.coef_

feature_names = X_train.columns

In [ ]:
feature_names

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'low quality',
       'sub_moderate quality', 'average quality', 'sub_high quality',
       'high quality'],
      dtype='object')

In [ ]:
coefficients

array([[-3.35573817e+04, -4.91435810e+03,  2.28534681e+02,
        -4.56380921e-01,  1.27199839e+04,  5.62436280e+05,
         6.43234852e+04,  5.17087732e+04, -5.86393761e+04,
        -1.57409424e+05, -1.44574367e+05, -7.06590489e+04,
         4.31282216e+05]])

The following is a list for all the variables with their corresponding coefficients:

'bedrooms': -33557.3817;

'bathrooms': -4914.35810;

'sqft_living': 228.534681

'sqft_lot': -0.456380921

'floors': 12719.9839

'waterfront': 562436.280

'view': 64323.4852

'condition': 51708.7732

'low quality': -58639.3761

'sub_moderate quality':  -157409.424

'average quality': -144574.367

'sub_high quality': -70659.0489

'high quality': 431282.216

##### In general, the larger the absolute value of the coefficient, the greater the impact of the drivers on house prices.

Accoding to the coefficients, these are the main drivers that should be getting more focus. They are the following:
1. number of bedrooms
2. number of floors
3. the waterfront view
4. the view of the property
5. condition of the house
6. the construction and design quality of the property

##### Results to the business executive:
1. For each additional floor in the number of floors, the average price of a house will increase by more than $10,000.
2. If the house is located overlooking the waterfront, the average price of the house will increase by more than $560,000.
3. If the house has a great view, the average value of the house increases by $64,000 for each point increases from 1 to 4 points.
4. If the house is in good condition, the average value of the house will increase by $50,000 for each point increases from 1 to 5.
5. If the design and construction level and quality of the house is very high, then the price of the house will be very high. For high quality houses, for each point increases from 11 to 13, their average price will go up by $400,000. 

However the exact opposite is the case for homes that are not of the highest quality in terms of design level and architectural style, and their home prices are usually very low. For every point from 10 to 1, there is a precipitous drop in home prices.